In [1]:
import numpy as np
from tkinter import *
import tkinter.scrolledtext as tkst
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.animation import FuncAnimation
from keras import optimizers
from keras.layers import Dense
from keras.models import Sequential
from keras.datasets import mnist # 손글씨 인식 데이터 로딩식 데이터 로딩

In [2]:
digit_fig = Figure(figsize=(4,4), dpi=100)
digit_ax = digit_fig.add_subplot(111)
digit_ax.set_title('Selected Digit Image')
# 격자 및 눈금을 만들기 위한 배열 생성
major_ticks = np.arange(0, 29, 5) # 주 눈금을 위한 배열 생성(단위 5)
minor_ticks = np.arange(0, 29, 1) # 보조 눈금을 위한 배열생성(단위 1)
# 만들어놓은 배열로 눈금 설정
digit_ax.set_xticks(major_ticks) # 주 눈금
digit_ax.set_xticks(minor_ticks, minor=True) # 보조눈금
digit_ax.set_yticks(major_ticks) # 주 눈금
digit_ax.set_yticks(minor_ticks, minor=True) # 보조눈금
digit_ax.grid(which='both') # 격자(grid) 생성(which='both' : 주, 보조 눈금 모두)
digit_ax.grid(which='minor', alpha=0.2) # 주눈금의 격자 투명도
digit_ax.grid(which='major', alpha=0.5) # 보조눈금의 격자 투명도

In [3]:
# loss(코스트)의 변화표시 차트
loss_fig = Figure(figsize=(4,4), dpi=100)
loss_ax = loss_fig.add_subplot(111)
loss_ax.set_xlabel('Epochs')
loss_ax.set_ylabel('Loss')
loss_ax.set_title('History of loss')

Text(0.5, 1.0, 'History of loss')

In [4]:
# 정확도 변화 표시 차트
acc_fig = Figure(figsize=(4,4), dpi=100)
acc_ax = acc_fig.add_subplot(111)
acc_ax.set_xlabel('Epochs')
acc_ax.set_ylabel('acc')
acc_ax.set_title('History of acc')

Text(0.5, 1.0, 'History of acc')

In [5]:
# 예측한 숫자에 대한 정확도(10개의 확률값) 표시 차트
number_fig = Figure(figsize=(4,4), dpi=100)
number_ax = number_fig.add_subplot(111)
number_ax.set_title('Probalility Chart')

Text(0.5, 1.0, 'Probalility Chart')

In [6]:
(x_train, y_train), (x_test, y_test) = mnist.load_data()
#print(x_train.shape)
#print(x_test.shape)

In [7]:
y_train[0]

5

In [8]:
def viewImage():
    # 데이터를 로드
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    # x_test 데이터의 원본을 x_test_original 변수에 복사
    x_test_original = x_test
    # 스핀박스에 입력된 숫자를 추출하고 -1 연산 : 배열은 0부터 시작하기 때문
    t_a = int(t_aSpbox.get())-1
    # x_test_original 배요ㅕㄹ의 selected_image 번째 위치한 데이터를 그래프에 드로잉
    # 데이터는 28*28 규격의 숫자 0~255사이의 채도가 저장되어있음
    selected_image=t_a
    digit_ax.imshow(x_test_original[selected_image])
    digit_fig.canvas.draw()    

In [9]:
def learning():
    import numpy as np
    from keras import optimizers
    from keras.layers import Dense
    from keras.models import Sequential
    # 데이터로드
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_test_original = x_test
    # 스핀박스 데이터 추출
    t_a = int(t_aSpbox.get()) -1
    t_t = int(t_tSpbox.get())
    selected_image = t_a
    
    # 모델 객체 생성
    model = Sequential()
    # relu 레이어 추가, 입력은 28*28 : (28, 28)데이터가 28*28로 변경예정, 출력은 512
    model.add(Dense(512, activation='relu', input_shape=(28*28,)))
    # softmax 레이어 추가. 출력 10개 - 
    # 전체출력을 원핫 구조로 설정된 개수만큼 변형해서 출력하는 레이어
    model.add(Dense(10, activation='softmax'))
    # 모델설계
    model.compile(optimizer='rmsprop', loss='categorical_crossentropy', \
                                                         metrics=['accuracy'])
    # 데이터 가공 : 원래 형태가 (60000, 28, 28) 이것을 입력 레이어 형태에 맞게 변형
    x_train = x_train.reshape((60000, 28*28))
    # 숫자가 0~255 사이에 있기 때문에 모두 1의자리와 소수점 아래 자리를 갖는
    # 데이터로 변형 - 학습 데이터와 테스트데이터들의 표준화 과정
    x_train = x_train.astype('float32') / 255
    x_test = x_test.reshape((10000, 28*28))
    x_test = x_test.astype('float32') / 255
    
    from keras.utils import to_categorical
    # 타겟값들을 케라스에서 제공하는 함수를 사용하여 원핫 형태로 인코딩합니다
    y_train = to_categorical(y_train)
    y_test = to_categorical(y_test)
    
    # 학습된 모델저장 ModelCheckpoint
    # 학습중 학습결과가 개선되지 않는 상태 지속 시 조기종료를 위한 EarlyStopping
    # import
    from keras.callbacks import EarlyStopping, ModelCheckpoint
    
    # ModelCheckpoint와 EarlyStopping의 객체를 생성
    modelCheckpoint = ModelCheckpoint('best_digits_model.hdf5', save_best_only=True)
    monitor_val_acc = EarlyStopping(monitor='val_accuracy', patience=10)
    # 'best_digits_model.hdf5' : 저장될 모델의 이름
    # monitor = 'val_accuracy' : 정확도를 이용한 모델학습 조기종료
    # patience =5 : 5회 학습이상 정확도가 개선되지 않으면 종료
    
    # 학습
    history = model.fit(x_train, y_train, validation_data=(x_test, y_test), \
            epochs=t_t, batch_size=128, callbacks=[monitor_val_acc, modelCheckpoint])
    
    # 예측
    result = model.predict(np.array([x_test[selected_image]]))
    #가장 높은 확률의 인덱스를 얻어내기 위해 argmax 함수 실행
    result_number = np.argmax(result)
    
    # 그래프 드로잉
    loss_ax.plot(history.history['loss'], 'ro', label='train loss')
    loss_fig.canvas.draw()
    
    acc_ax.plot(history.history['accuracy'], 'bo', label='train acc')
    acc_fig.canvas.draw()
    
    # 표시 라벨들
    digits = ('0','1','2','3','4','5','6','7','8','9')
    x_pos = np.arange(len(digits)) # x축 값들
    # 차트에 표시될 10개의 확률들을 리스트로 저장
    performance=[ val for val in result[0] ]
    
    # 10개의 확률 중 가장 큰 확률값의 인덱스를 갖고 있는 result_number 변수 이용
    # 가장 큰 확률(예측 숫자의 확률)을 추출
    result_probability = performance[result_number]
    # 계산해 둔 x_pos(x축 데이터)와 performance(y축 데이터)로 차트를 그립니다
    number_ax.bar(x_pos, performance, align='center', alpha=0.5)
    
    # 차트의 타이틀에 예측한 숫자와 그의 확률값을 표시합니다.
    number_ax.set_title('Number is %2i (probability %7.4f)' % (result_number, \
                                                          result_probability*100))
    number_fig.canvas.draw()
    # 열 개의 확률 출력
    print(performance)

In [12]:
# 윈도우 생성
main = Tk()
main.title("MNIST Digits, Neural Network")
main.geometry()

t_aVal = IntVar(value=1)
t_aSpbox = Spinbox(main, textvariable=t_aVal, from_=0, to=10000, increment=1)
t_aSpbox.grid(row=0, column=1)
t_aLabel=Label(main, text='Number of Digits : ')
t_aLabel.grid(row=0, column=0)
# 스핀박스에 선택한 숫자를 인덱스로 10000개의 test 데이터 중 입력된 인덱스번째의 이미지를
# 불러오기 위한 버튼 - viewImage() 함수에 연결

btn1 = Button(main, text="viewImage", height=1,command=lambda:viewImage())
btn1.grid(row=0, column=2, sticky=(W,E))

# train 횟수를 지정하는 스핀박스와 라벨
t_tVal = IntVar(value=100)
t_tSpbox = Spinbox(main, textvariable=t_tVal, from_=0, to=100000, increment=100)
t_tSpbox.config(state='readonly')
t_tSpbox.grid(row=0, column=4)
t_tLabel=Label(main, text='Number of trains : ')
t_tLabel.grid(row=0, column=3)

# 스크롤텍스트와 스타일 설정
digit_canvas = FigureCanvasTkAgg(digit_fig, main)
digit_canvas.get_tk_widget().grid(row=1, column=0, columnspan=3)

btn2 = Button(main, text="Deep Learning", height=1,command=lambda:learning())
btn2.grid(row=0, column=5, sticky=(W,E)) 

loss_canvas = FigureCanvasTkAgg(loss_fig, main)
loss_canvas.get_tk_widget().grid(row=1,column=3,columnspan=3)

acc = FigureCanvasTkAgg(acc_fig, main)
acc.get_tk_widget().grid(row=2,column=0,columnspan=3)

number = FigureCanvasTkAgg(number_fig, main)
number.get_tk_widget().grid(row=2,column=3,columnspan=3)

main.mainloop()